<a href="https://colab.research.google.com/github/colinmcnamara/austin_langchain/blob/main/LC_102-LangServe-pirate-speak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install packages
* langchain-cli
* Additional dependencies (added manually due to dependency manager issue)


In [1]:
%pip install -q langchain-cli kaleido python-multipart tensorflow-probability

# Set environment variables from Google Colab Secrets
* Enable notebook access for each key you want to use
* Highlight each line "CMD+SHIFT+/" to comment/uncomment # a line


---



# Exporting variables from Colab key manager into the notebook AND shell
* This takes the keys from your colab instance, maps them into your notebook AND creates a shell script you can execute from your terminal to set all of your keys


In [2]:
import os
from google.colab import userdata

# List of your secret names, one per line for easy commenting
secret_names = [
    'OPENAI_API_KEY',
    # 'SERPAPI_API_KEY',
    # 'PINECONE_API_KEY',
    # 'PINECONE_ENV',
    # 'GOOGLE_CSE_ID',
    # 'HUGGINGFACEHUB_API_TOKEN',
    # 'COHERE_API_KEY',
    # 'GOOGLE_API_KEY',
    'LANGCHAIN_TRACING_V2', # Uncomment when needed
    'LANGCHAIN_ENDPOINT', # Uncomment when needed
    'LANGCHAIN_API_KEY' # Uncomment when needed
]

# Start the shell script with the shebang line
shell_script = "#!/bin/sh\n"

# Loop through each secret name and set the corresponding environment variable
for secret_name in secret_names:
    # Retrieve the secret value from Colab's secret manager
    secret_value = userdata.get(secret_name)

    # Check if the secret value was successfully retrieved
    if secret_value is not None:
        # Set the secret as an environment variable in Python
        os.environ[secret_name] = secret_value
        print(f"Environment variable '{secret_name}' set in Python environment.")

        # Add the secret to the shell script for the terminal environment
        shell_script += f"export {secret_name}='{secret_value}'\n"
        shell_script += f'echo "Environment variable \'{secret_name}\' set in terminal environment."\n'
    else:
        print(f"Secret '{secret_name}' not found. Please check the key name and try again.")

# Uncomment the lines below to enable keys in your shell
# to a shell file to use langchain-cli or langserve
# be sure to delete your instance after so you don't leave keys lying around
#
# Write the shell script to a file
with open('set_env_vars.sh', 'w') as file:
    file.write(shell_script)

# Make the script executable
!chmod +x set_env_vars.sh

print("Shell script 'set_env_vars.sh' created and made executable. You can source it in your terminal with 'source set_env_vars.sh'. It will display which variables have been set.")


Environment variable 'OPENAI_API_KEY' set in Python environment.
Environment variable 'LANGCHAIN_TRACING_V2' set in Python environment.
Environment variable 'LANGCHAIN_ENDPOINT' set in Python environment.
Environment variable 'LANGCHAIN_API_KEY' set in Python environment.
Shell script 'set_env_vars.sh' created and made executable. You can source it in your terminal with 'source set_env_vars.sh'. It will display which variables have been set.


# Verify your OPENAI_API_KEY variable is set
* Scroll down to verify your OPENAI_API_KEY is set


In [3]:
import os

if 'OPENAI_API_KEY' in os.environ:
    print("OPENAI_API_KEY is set, but we're not displaying the value")
else:
    print("OPENAI_API_KEY is unset or set to the empty string")


OPENAI_API_KEY is set, but we're not displaying the value


## Open your terminal
* Click icon  [>_] on the lower left
* Terminal should appear on your right
* Type the following, then hit enter
```
source set_env_vars.sh
```

## In your terminal -->
## source set_env_vars.sh
```
source set_env_vars.sh
```
* you should see notes saying the variables you specified are set:

* example output:
```
Environment variable 'OPENAI_API_KEY' set in terminal environment.
```

# validate the following commands are available in the terminal
* type **lang** and hit tab twice to see what commands start with **lang**
```
lang
```
expected output
```
/content/my-app# lang
langchain         langchain-cli     langchain-server  langsmith
```

# Create a new LangChain project
Create a new app named my app
```
langchain app new my-app
```
list the files inside of the folder my-app
```
ls my-app
```

* view that the following directores are created
  * app - LangServe code
  * packages - Chains and Agents




Expected output:
```
/content/my-app# ls
app  logs  packages  pyproject.toml  README.md
```

#install packages for pirate-speak

In [4]:
%pip install -q cohere tiktoken

# Add templates into your project
 Change directory into **my-app**
```
cd my-app
```
add the pirate-speak template to your app
```
langchain app add pirate-speak
```
Answer '**y** to the run it question, and hit enter
```
"Run it? [y/N]:"
```
you should see a lot of output as it installs. If you see any pip errors, adjust the packages list above to resolve the issues
```
Successfully installed openai-0.28.1 pirate-speak-0.0.1
```

# Generate Route code
* Generate route code for these packages? [Y/n]: (answer y)

It will generate the following code that we need to add to server.py

Great! Add the following to your app: in content/my-app/app/server.py by
* navigating to file manager on your left hand side (under the key)
* Click on **my-app** / **app** / to expand the directories
* Double-Click on **server.py**
* A new tab should open next to your terminal on the right. This is that file

import your pirate_speak_chain by addeding a from statement
```
from pirate_speak.chain import chain as pirate_speak_chain
```

Replace this -
```
add_routes(app, NotImplemented)
```
with your new chain routes connecting your app, the pirate speak chain, and specifying the path to the folder (this is important since we can edit this late)
```
add_routes(app, pirate_speak_chain, path="/pirate-speak")
```
* hit **ctrl+s** to save (or click on file>save)

Your new server.py file should look like this
```
from fastapi import FastAPI
from langserve import add_routes
from pirate_speak.chain import chain as pirate_speak_chain

app = FastAPI()

# Edit this to add the chain you want to add
add_routes(app, pirate_speak_chain, path="/pirate-speak")

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)
```

# Start langchain server!!!
Type the command below. This will start up LangServe, and load the app as a web service using FASTAPI
```
langchain serve
```
You should see something like this:
```
/content/my-app# langchain serve
INFO:     Will watch for changes in these directories: ['/content/my-app']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [8879] using StatReload
INFO:     Started server process [8888]
INFO:     Waiting for application startup.

 __          ___      .__   __.   _______      _______. _______ .______     ____    ____  _______
|  |        /   \     |  \ |  |  /  _____|    /       ||   ____||   _  \    \   \  /   / |   ____|
|  |       /  ^  \    |   \|  | |  |  __     |   (----`|  |__   |  |_)  |    \   \/   /  |  |__   
|  |      /  /_\  \   |  . `  | |  | |_ |     \   \    |   __|  |      /      \      /   |   __|  
|  `----./  _____  \  |  |\   | |  |__| | .----)   |   |  |____ |  |\  \----.  \    /    |  |____
|_______/__/     \__\ |__| \__|  \______| |_______/    |_______|| _| `._____|   \__/     |_______|

LANGSERVE: Playground for chain "/pirate-speak" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /pirate-speak/playground
LANGSERVE:
LANGSERVE: See all available routes at /docs

INFO:     Application startup complete.
```

# Getting to the interface through Google Colab
Since we are in a hosted google envoironment, we are going to have to do a couple tricks to access the API / Playground

* We are going to have to tunnel our way out using npx


## Find the IP of your instance

In [7]:
!curl ipv4.icanhazip.com
!echo "Copy this IP into the webpage that opens below"

34.106.243.223
Copy this IP into the webpage that opens below


## Expose the Streamlit app on port 8501

In [ ]:
!npx localtunnel --port 8000

npx: installed 22 in 4.464s
your url is: https://khaki-coins-type.loca.lt


#Authenticate through you the web interface
* Paste your ip in
* Hit enter

#Add your app uri
* https://your_url/pirate-speak/playground

Now play with the web interface, interact, as it some questions


# Modify your applications code
* In the folder interface on your left click on
  * my-app
    * packages
      * pirate-speak
        * chain.py


You should notice the chain.py file open up next to your terminal tab
* Click it to edit the file and change pirate speak to elite haX0r speak
```
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
```
```
_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
#           "translate user input into pirate speak", # old line
            "Translate user input into elite haX0r speak", # new line
        ),
        ("human", "{text}"),
    ]
)
_model = ChatOpenAI()
```
```
# if you update this, you MUST also update ../pyproject.toml
# with the new `tool.langserve.export_attr`
chain = _prompt | _model
```

# Press save, and click over to the terminal
* Notice that LangServe picked up the change hwen you saved your file

```
WARNING:  StatReload detected changes in 'packages/pirate-speak/pirate_speak/chain.py'. Reloading...
/content/my-app# INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [17853]
INFO:     Started server process [17869]
INFO:     Waiting for application startup.

 __          ___      .__   __.   _______      _______. _______ .______     ____    ____  _______
|  |        /   \     |  \ |  |  /  _____|    /       ||   ____||   _  \    \   \  /   / |   ____|
|  |       /  ^  \    |   \|  | |  |  __     |   (----`|  |__   |  |_)  |    \   \/   /  |  |__   
|  |      /  /_\  \   |  . `  | |  | |_ |     \   \    |   __|  |      /      \      /   |   __|  
|  `----./  _____  \  |  |\   | |  |__| | .----)   |   |  |____ |  |\  \----.  \    /    |  |____
|_______/__/     \__\ |__| \__|  \______| |_______/    |_______|| _| `._____|   \__/     |_______|

LANGSERVE: Playground for chain "/pirate-speak" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /pirate-speak/playground
LANGSERVE:
LANGSERVE: See all available routes at /docs

INFO:     Application startup complete.
```

# go back to your playground, and interact with this new application